<h1> Titanics Dataset Predictions </h1>
<h2> Jose M. Serra Jr. </h2>
<p> In this project I will be conducting a deep learning excercise where I will be using a Support Vector Machine to predict the people that will be saved on the Titanic from the Knowledge Titanic Dataset.

In [1]:
import os
source = r'C:\Users\joses\source\repos\Kaggle_Titanic_Artifical_Intelligence'
os.chdir(source)

In [2]:
!echo %cd%

C:\Users\joses\source\repos\Kaggle_Titanic_Artifical_Intelligence


In [3]:
import matplotlib.pyplot as mplplt
import numpy as np
import pandas as pd
from scipy.stats import reciprocal
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.utils import resample

In [4]:
file1_train, file2_test = pd.read_csv(r"train.csv", delimiter=","),\
pd.read_csv(r"test.csv", delimiter= ",")

In [5]:
def func(df):
  return pd.concat([df.head(),df.tail()])

In [6]:
func(file1_train)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.7500,NaN,Q


In [7]:
func(file2_test)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


<p> Now I am going to encode the Sex column into a binary 1s, and 0s output Sex_Binary column.</p>

In [8]:
X, y = file1_train.drop("Survived", axis=1), file1_train["Survived"]

In [9]:
objs = X.select_dtypes(["object"])
num  = X.select_dtypes(["number"])

In [10]:
objs.isnull().sum().head()

Name          0
Sex           0
Ticket        0
Cabin       687
Embarked      2
dtype: int64

In [11]:
num.isnull().sum().head()

PassengerId      0
Pclass           0
Age            177
SibSp            0
Parch            0
dtype: int64

In [12]:
def func_K1(index_arr):
  return 10 if index_arr == 1 else ( 5 if index_arr == 2 else 0)

In [13]:
Target = file2_test.PassengerId

In [14]:
X.PassengerId = X.PassengerId.apply(func=func_K1)

In [15]:
file2_test.PassengerId = file2_test.PassengerId.apply(func=func_K1)

In [16]:
X, file2_test = X.drop('PassengerId', axis=1), file2_test.drop('PassengerId',axis=1)

In [17]:
num  = X.select_dtypes(["number"])

In [18]:
num.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [19]:
def func_K2(K):
  a = {"G": 0, "F":5, "E": 10,
       "D": 15, "C":20, "B":25,
       "A":30, "T":40}
  return a.get(K)

In [20]:
X.Cabin = X.Cabin.fillna("C").apply(lambda x: x[0]).apply(func=func_K2).astype("int32")

In [21]:
file2_test.Cabin = file2_test.Cabin.fillna("C").apply(lambda x: x[0]).apply(func=func_K2).astype("int32")

In [22]:
objs = X.select_dtypes(["object"])
num  = X.select_dtypes(["number"])

In [23]:
num.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin'], dtype='object')

In [24]:
numerical_features = num.columns
numerical_pipeline = Pipeline(
    steps=[
            ("imputer", SimpleImputer(strategy = 'mean')),
            ("scaler", StandardScaler())
])

In [25]:
category_counts = X.Name.apply(func=lambda x: x.split(' ')[1]).value_counts()
category_weights = {category: count for category, count in category_counts.items()}

# Map the weights to the original DataFrame
series1 = X.Name.apply(func=lambda x: x.split(' ')[1]).map(category_weights)

# Normalize the weights based on the number of rows
total_rows = len(X)
X['Normalized_Weight'] = series1 / total_rows

In [26]:
X = X.drop('Name', axis=1)

In [27]:
num  = X.select_dtypes(["number"])

In [28]:
num.isnull().sum().head()

Pclass      0
Age       177
SibSp       0
Parch       0
Fare        0
dtype: int64

In [29]:
for i in range(len(objs.columns)):
    print(objs.columns[i], objs.iloc[:,i].value_counts().unique())

Name [1]
Sex [577 314]
Ticket [7 6 5 4 3 2 1]
Embarked [644 168  77]


In [30]:
binary = ["Sex"]
binary_pipeline = Pipeline(steps=[("binary", OneHotEncoder())])

In [31]:
#cat1 = ["Ticket", "Cabin", "Embarked"]
cat1 = ["Embarked"]
catergorical_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy = 'most_frequent')), ("ordinal_encoder", OrdinalEncoder()),("scaler", StandardScaler())])

In [32]:
data_preprocessor = ColumnTransformer( [('numerical', numerical_pipeline, numerical_features),
                                      ('binary', binary_pipeline, binary),
                                      ('categorical', catergorical_pipeline, cat1)])

In [33]:
func(X)

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Normalized_Weight
0,3,male,22.0,1,0,A/5 21171,7.2500,20,S,0.563412
1,1,female,38.0,1,0,PC 17599,71.2833,20,C,0.135802
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,20,S,0.200898
3,1,female,35.0,1,0,113803,53.1000,20,S,0.135802
4,3,male,35.0,0,0,373450,8.0500,20,S,0.563412
886,2,male,27.0,0,0,211536,13.0000,20,S,0.006734
887,1,female,19.0,0,0,112053,30.0000,25,S,0.200898
888,3,female,NaN,1,2,W./C. 6607,23.4500,20,S,0.200898
889,1,male,26.0,0,0,111369,30.0000,20,C,0.563412
890,3,male,32.0,0,0,370376,7.7500,20,Q,0.563412


In [34]:
beta = X.columns.to_list()
beta.remove('Ticket')


In [35]:
X = Pipeline(steps=[('processing',data_preprocessor)]).fit_transform(X[beta])

Support Vector Machine Results Tabulated with Gamma, Accuracy, Kernel Comparison
$$\begin{array}{|c|c|c|}
\hline
\mathbf{Gamma}&\mathbf{Accuracy}&\mathbf{Kernel}\\
\hline
1.0&.76794&\mathbf{RBF}\\
2.0&.77511&\mathbf{RBF}\\
3.0&.76794&\mathbf{RBF}\\
1.0&.64832&\mathbf{Sigmoid}\\
2.0&.65071&\mathbf{Sigmoid}\\
3.0&.65071&\mathbf{Sigmoid}
\end{array}$$



In [36]:
from sklearn.svm import SVC
clf = SVC() #Refer to table for params

In [37]:
_ = clf.fit(X=X,y=y)

In [38]:
X_test = data_preprocessor.fit_transform(file2_test)

In [39]:
PassengerId = Target.to_list()

In [40]:
final_pred = clf.predict(X=X_test)

In [41]:
d = {"PassengerId":PassengerId, "Survived":final_pred}
pd.DataFrame(data=d,index=None, columns= ["PassengerId","Survived"]).to_csv("12212023.csv",index=False, header=1)